## 🤖 AutoGen – RoundRobin Multi-Agent Collaboration (Basic Version)

### ✅ Objective:
Simulate a collaborative coding session using **AutoGen's `RoundRobinGroupChat`**, where each agent contributes to a Python code solution refinement loop.

---

### 🧩 Agent Roles:

| Agent Name         | Description |
|--------------------|-------------|
| `complex_dsa_solver` | Generates the initial Python solution to a DSA problem |
| `code_reviewer`      | Reviews and briefly comments on code optimization (max 10 words) |
| `code_editor`        | Simplifies and annotates the code (adds comments, improves clarity) |

---

### 🧪 Task Given:
```text
Write a simple code in python to add two numbers

In [ ]:
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    raise ValueError('GEMINI_API_KEY is missing')

llm_client = OpenAIChatCompletionClient(model="gemini-1.5-flash-8b", api_key=api_key)

dsa_solver = AssistantAgent(
    name='complex_dsa_solver',
    model_client=llm_client,
    description='A DSA solver',
    system_message='You give code in Python to solve the complex DSA problem. Give under 100 words.'
)

code_reviewer = AssistantAgent(
    name='code_reviewer',
    model_client=llm_client,
    description='A code reviewer',
    system_message="You review the code given by complex_dsa_solver and make sure it's optimized. Give under 10 words."
 )

code_editor = AssistantAgent(
    name='code_editor',
    model_client=llm_client,
    description='A code editor',
    system_message="You make the code easy to understand and add comments wherever required. Give under 10 words."
)


team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=10
)

async def run_team():
    text = TextMessage(content="Write a simple code in python to add two numbers", source='user')

    result = await team.run(task=text)

    for msg in result.messages:
        print(f'\n----- {msg.source} -----')
        # print(msg.source)
        print(msg.content)

await run_team()


----- user -----
Write a simple code in python to add two numbers

----- complex_dsa_solver -----
```python
def add_numbers(x, y):
  """Adds two numbers."""
  return x + y

# Example usage
num1 = 10
num2 = 5
sum_result = add_numbers(num1, num2)
print(f"The sum of {num1} and {num2} is: {sum_result}")
```


----- code_reviewer -----
```python
def add_numbers(x, y):
  return x + y
```


----- code_editor -----
Adds two numbers.


----- complex_dsa_solver -----
```python
def add_numbers(x, y):
  """Adds two numbers together."""
  return x + y
```


----- code_reviewer -----
This code is already optimized.


----- code_editor -----
OK.


----- complex_dsa_solver -----
Okay.  Is there anything else I can help you with?


----- code_reviewer -----
Yes, please provide me with a task or problem you would like me to help you with.


----- code_editor -----
Write a Python function that takes a list of numbers and returns the sum of all the even numbers in the list.  For example, if the input lis

#### ✅ What We Achieved

You successfully created an **AutoGen RoundRobin multi-agent team** using:

- `complex_dsa_solver` (code writer)
- `code_reviewer` (short reviewer)
- `code_editor` (clean-up and comments)

They correctly collaborated to solve your first task:  
👉 “**Write a simple code in Python to add two numbers**”

---

#### ⚠️ What’s the Issue

After completing the initial task, the agents **continued the conversation on their own**, generating a new, unrelated DSA task:  
👉 **Filtering even numbers from a list** — even though the user never asked for it.

---

#### 🎯 Root Cause

- You set `max_turns=10` without limiting the conversation
- Missing the config: `stop_on_user_message=True`
- Agents tried to stay “proactive” by **inventing follow-up prompts**

## 🧠 AutoGen RoundRobin – Multi-Agent Code Refinement

### ✅ Objective:
Use **AutoGen v0.6.2+** to create a collaborative agent team that:
- Solves a given Python DSA problem
- Reviews and optimizes the solution
- Produces FAANG-grade, production-ready code

---

### 🧩 Agents Involved:
| Agent           | Role                                                                 |
|------------------|----------------------------------------------------------------------|
| `dsa_solver`     | Writes initial Python code to solve the problem                     |
| `code_reviewer`  | Reviews for logic, edge cases, performance, and correctness         |
| `code_editor`    | Finalizes the code with clean structure, type hints, comments, and tests |

---

In [ ]:
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination

load_dotenv()

os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')


llm_client = OpenAIChatCompletionClient(model="gemini-1.5-flash-8b", api_key=api_key)

dsa_solver_2 = AssistantAgent(
    name="dsa_solver",
    model_client=llm_client,
    description='You are a DSA solver',
    system_message="""You are a Python developer tasked with solving real-world DSA problems.  
    Always write clear, correct, and modular solutions that:

    - Include a docstring explaining the function
    - Handle edge cases and invalid input
    - Use meaningful variable names
    - Follow Pythonic conventions (PEP8)
    - Solve the problem efficiently

    Keep your code self-contained and under 15 lines. Avoid print statements unless required. Let reviewers optimize further if needed.
    """
)

code_reviewer_2 = AssistantAgent(
    name='code_reviewer',
    model_client=llm_client,
    description='You are a code reviewer.',
    system_message="You are a strict Python code reviewer. Review all code thoroughly for optimization, clarity, error handling, and real-world robustness. Only say 'TERMINATE' if the code meets production and FAANG-grade standards."
)

code_editor_2 = AssistantAgent(
    name='code_editor',
    model_client=llm_client,
    description='You are a code editor',
    system_message="""You are a FAANG-grade code editor and your responsibility is to ensure the final Python code is optimized, readable, robust, and production-ready. You must enforce high standards including:
    - Proper function and variable naming
    - Docstrings and inline comments
    - Type hints (if applicable)
    - Input validation and edge case handling
    - Minimal test or usage examples

    Only say 'TERMINATE' if the code meets all of the above standards and no further improvements are needed."""

)

custom_termination_msg = TextMentionTermination(text='TERMINATE')

team = RoundRobinGroupChat(
    participants=[dsa_solver_2, code_reviewer_2, code_editor_2],
    max_turns=10,
    termination_condition=custom_termination_msg
)

async def run_python_team():
    text = TextMessage(content="write a python code to add two numbers.", source='user')

    result = await team.run(task=text)

    for msg in result.messages:
        print(f"\n-----{msg.source}-----")
        print(msg.content)

await run_python_team()


-----user-----
write a python code to add two numbers.

-----dsa_solver-----
```python
def add_numbers(num1, num2):
    """Adds two numbers.

    Args:
        num1: The first number.
        num2: The second number.

    Returns:
        The sum of the two numbers.
        Returns an error message if input is not a number.
    """
    try:
        return num1 + num2
    except TypeError:
        return "Invalid input: Input must be numeric."
```


-----code_reviewer-----
```python
def add_numbers(num1, num2):
    """Adds two numbers.

    Args:
        num1: The first number.
        num2: The second number.

    Returns:
        The sum of the two numbers.
        Raises TypeError if input is not numeric.  Returns None if an unexpected error occurs.
    """
    try:
        if not isinstance(num1, (int, float)) or not isinstance(num2, (int, float)):
            raise TypeError("Invalid input: Input must be numeric.")
        return num1 + num2
    except TypeError as e:
        rais

## ✅ AutoGen RoundRobin Output – Code Review Summary

### 🧠 Objective:
Agents collaboratively refined a Python function to **add two numbers**, enforcing best practices through structured rounds.

---

### 🧩 Agent-Level Observations:

#### 🔹 `dsa_solver`
- Provided a basic working implementation using `try-except`
- Included a simple docstring
- Weak input validation (relies on catching `TypeError` after operation)

#### 🔹 `code_reviewer`
- Improved type checking with `isinstance()` for both inputs
- Added detailed exception handling (`TypeError` and general `Exception`)
- Used `raise` for explicit error propagation
- Introduced `None` return for graceful fallback in case of unexpected errors

#### 🔹 `code_editor`
- Finalized code to FAANG-grade level:
  - ✅ Added type hints (`float`)
  - ✅ Cleaned logic: removed redundant `try-except` after input validation
  - ✅ Strengthened docstring with `Raises` section
  - ✅ Included a usage/testing block covering:
    - Integer input
    - Float input
    - Invalid input (string + number)

---

### ✅ Key Strengths:
- Strong adherence to **PEP8 / Pythonic standards**
- Production-ready function with error handling and clear user messaging
- Example usage block makes the function testable and understandable
- Clean agent collaboration — each agent added measurable improvements
- Intelligent and correct termination with `TERMINATE`

---

### ❗ Optional Enhancements:
- Use `Union[int, float]` as input type hints if supporting both types explicitly:
  ```python
  from typing import Union
  def add_numbers(num1: Union[int, float], num2: Union[int, float]) -> float:


In [ ]:
from autogen_agentchat.base import TaskResult

team_2 = RoundRobinGroupChat(
    participants=[dsa_solver_2, code_reviewer_2, code_editor_2],
    max_turns=10,
    termination_condition=custom_termination_msg
)

async for msg in team_2.run_stream(task="write a simple Hello World Python code"):
    print(type(msg))

    if isinstance(msg, TaskResult):
        print(f"stop reason: {msg.stop_reason}")
    else:
        print(msg.source, msg.content)